#Random Forest


## Importación de librerías y datos

Por medio de nuestra libería ESIOS_contoller.py importamos nuestro último dataset de datos y lo parseamos para su uso. Sirve tanto como para Drive como jupiter.

In [0]:
import json, urllib, datetime, pickle, time
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import *
from sklearn.metrics import *
from keras.models import *
from keras.layers import *
from sklearn.preprocessing import *
from keras.optimizers import *
from scipy.stats import *
from importlib.machinery import SourceFileLoader


try:
  from google.colab import drive
  drive.mount('/content/drive')
  path = '/content/drive/My Drive/TFM/Utils/ESIOS_contoller.py'
  in_colab = True
except:
  path = '../utils/ESIOS_contoller.py'
  in_colab = False
  

esios_assembler = SourceFileLoader('esios', path).load_module()

esios_controller = esios_assembler.ESIOS(in_colab)
data_consumo = esios_controller.get_data('non-secuencial')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mostrando los datos de data_total_for_non_serial.csv
(30555, 33)
________________________________________________________________________________


## Preparación de los datos

In [0]:
x_data = esios_controller.get_data_real_time()
y_data = esios_controller.get_target_data()

x_data = x_data.drop(['PVPC-1h', 'PVPC-2h'], axis=1)

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.33, shuffle=False)

#Reshape for the LSTM
x_train = x_train.to_numpy()
x_valid = x_valid.to_numpy()
y_train = y_train.to_numpy()
y_valid = y_valid.to_numpy()

print('Xtrain_dim:', x_train.shape)
print('Ytrain_dim:', y_train.shape)

Xtrain_dim: (20471, 21)
Ytrain_dim: (20471, 1)


## Modelo

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

regressor = RandomForestRegressor(random_state=0, n_estimators=100)
regressor.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [0]:

cross_val_score(regressor, x_train, y_train, cv=10)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 

array([0.60364092, 0.62355775, 0.50070493, 0.70663584, 0.76277778,
       0.68592252, 0.19457789, 0.66830145, 0.75727435, 0.70753283])

In [0]:
print('R2 en entrenamiento es: ', regressor.score(x_train, y_train))
print('R2 en validación es: ', regressor.score(x_valid, y_valid))

print('MAE: ', mean_absolute_error(regressor.predict(x_valid), y_valid))
print('MSE: ', mean_squared_error(regressor.predict(x_valid), y_valid))
#print('RMSE: ', mean_squared_log_error(regressor.predict(x_valid), y_valid))
print('Variance: ', explained_variance_score(regressor.predict(x_valid), y_valid))
print('R2: ', r2_score(regressor.predict(x_valid), y_valid))
#print(regressor.coef_)

R2 en entrenamiento es:  0.9883126652941389
R2 en validación es:  0.7567186876077785
MAE:  4.1860113942879815
MSE:  30.802442505544438
Variance:  0.6643209017899442
R2:  0.6631379378214339


## Normalizados

In [0]:
scaler = StandardScaler()
x_train_est = scaler.fit_transform(x_train)
y_train_est = scaler.fit_transform(y_train)

In [0]:
regressor_standarized = RandomForestRegressor(random_state=0, n_estimators=100)
regressor_standarized.fit(x_train_est, y_train_est)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [0]:
print('R2 en entrenamiento es: ', regressor_standarized.score(x_train_est, y_train_est))
print('R2 en validación es: ', regressor_standarized.score(x_valid, y_valid))

print('MAE: ', mean_absolute_error(regressor_standarized.predict(x_valid), y_valid))
print('MSE: ', mean_squared_error(regressor_standarized.predict(x_valid), y_valid))
#print('RMSE: ', mean_squared_log_error(regressor_standarized.predict(x_valid), y_valid))
print('Variance: ', explained_variance_score(regressor_standarized.predict(x_valid), y_valid))
print('R2: ', r2_score(regressor_standarized.predict(x_valid), y_valid))

R2 en entrenamiento es:  0.9887086550119095
R2 en validación es:  -111.97693548216098
MAE:  119.07033564794561
MSE:  14304.286364714353
Variance:  -4189.062443967993
R2:  -473644.69935415254


## Optimización de modelos

In [0]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [10, 100, 150, 200, 300, 500, 600, 750, 1000]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

#param_grid = {'n_estimators': n_estimators}
#param_grid = {'max_features': max_features}
#param_grid = {'max_depth': max_depth}
#param_grid = {'min_samples_split': min_samples_split}
#param_grid = {'min_samples_leaf': min_samples_leaf}
param_grid = {'bootstrap': bootstrap, 
             'min_samples_leaf': min_samples_leaf}

clf = GridSearchCV(regressor, param_grid, cv=2)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 

GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=0,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'bootstrap': [True, False],
         

In [0]:
 print(clf.best_params_)

{'bootstrap': True, 'min_samples_leaf': 4}


* {'n_estimators': 1000}
* {'max_features': 'auto'}
* {'max_depth': 20}
* {'min_samples_split': 2}
* {'min_samples_leaf': 4}
* {'bootstrap': True}



In [0]:
regressor_optimun = RandomForestRegressor(
    random_state=0,
    n_estimators=300,
    max_features='auto',
    max_depth=20,
    min_samples_split=2,
    min_samples_leaf=4,
    bootstrap=True
)
regressor_optimun.fit(x_train, y_train)
cross_val_score(regressor_optimun, x_train, y_train, cv=5)   
print('R2 en entrenamiento es: ', regressor_optimun.score(x_train, y_train))
print('R2 en validación es: ', regressor_optimun.score(x_valid, y_valid))

print('MAE: ', mean_absolute_error(regressor_optimun.predict(x_valid), y_valid))
print('MSE: ', mean_squared_error(regressor_optimun.predict(x_valid), y_valid))
#print('RMSE: ', mean_squared_log_error(regressor_optimun.predict(x_valid), y_valid))
print('Variance: ', explained_variance_score(regressor_optimun.predict(x_valid), y_valid))
print('R2: ', r2_score(regressor_optimun.predict(x_valid), y_valid))
#print(regressor.coef_)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was

R2 en entrenamiento es:  0.9717396093155645
R2 en validación es:  0.7707067622380049
MAE:  4.056815060789081
MSE:  29.031378134327287
Variance:  0.6755099806309264
R2:  0.6748511960893852


In [0]:
regressor_optimun = RandomForestRegressor(
    random_state=0,
    n_estimators=1000,
    max_features='auto',
    max_depth=20,
    min_samples_split=2,
    min_samples_leaf=4,
    bootstrap=True
)
regressor_optimun.fit(x_train_est, y_train_est)
cross_val_score(regressor_optimun, x_train_est, y_train_est, cv=5)   
print('R2 en entrenamiento es: ', regressor_optimun.score(x_train_est, y_train_est))
print('R2 en validación es: ', regressor_optimun.score(x_valid_est, y_valid_est))

print('MAE: ', mean_absolute_error(regressor_optimun.predict(x_valid_est), y_valid_est))
print('MSE: ', mean_squared_error(regressor_optimun.predict(x_valid_est), y_valid_est))
#print('RMSE: ', mean_squared_log_error(regressor_optimun.predict(x_valid_est), y_valid_est))
print('Variance: ', explained_variance_score(regressor_optimun.predict(x_valid_est), y_valid_est))
print('R2: ', r2_score(regressor_optimun.predict(x_valid_est), y_valid_est))
#print(regressor.coef_)

NameError: ignored

## Visualización del modelo

In [0]:
# Extract single tree
estimator = regressor.estimators_[5]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = x_valid_est.feature_names,
                class_names = y_valid_est.target_names,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

AttributeError: ignored